In [23]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [42]:
# Angles (3) + Accel (3) + Compass(3)
#TODO 2 imus in so double data width?
#data_input_width = 3 + 3 + 3
data_input_width = 3 #use only angles for now
time_steps= 10

# Input Format
# [angles(t-time_steps+1), accel(t-time_steps+1), compass(t-time_steps+1)]
# ...
# [angles(t-2), accel(t-2), compass(t-2)]
# [angles(t-1), accel(t-1), compass(t-1)]
# [angles(t) accel(t), compass(t)]

#output Format
#
# [correct_exercise, overcompensation]
#

In [43]:
model = keras.Sequential()
#TODO I don't know if simpleRNN should come before or after LSTM
#TODO another input second IMU?
model.add(layers.SimpleRNN(32, input_shape = (time_steps, data_input_width), return_sequences=True))
model.add(layers.LSTM(16))
model.add(layers.Dense(10))
model.add(layers.Dense(2, activation='sigmoid')) #sigmoid has output between 0 and 1 (confidence)
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_19 (SimpleRNN)   (None, 10, 32)            1152      
                                                                 
 lstm_6 (LSTM)               (None, 16)                3136      
                                                                 
 dense_34 (Dense)            (None, 10)                170       
                                                                 
 dense_35 (Dense)            (None, 2)                 22        
                                                                 
Total params: 4,480
Trainable params: 4,480
Non-trainable params: 0
_________________________________________________________________


In [ ]:
N = 16    
Tp =12 

t=np.arange(0,N)
x=np.sin(0.02*t) #+2*np.random.rand(N)
df = pd.DataFrame(x)
df.head()

values=df.values
train,test = values[0:Tp,:], values[Tp:N,:]
step = 4
test = np.append(test,np.repeat(test[-1,],step))
train = np.append(train,np.repeat(train[-1,],step))

def convertToMatrix(data, step):
    X, Y =[], []
    for i in range(len(data)-step):
        d=i+step  
        X.append(data[i:d,])
        Y.append(data[d,])
    return np.array(X), np.array(Y)

trainX,trainY =convertToMatrix(train,step)
testX,testY =convertToMatrix(test,step)

trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

print(trainX)
print(trainY)


In [52]:
def read_format_csv(filename):
    try:
        data = pd.read_csv(filename)
    except Exception:
        raise Exception
    data.pop("time")
    data.pop("IMU#")
    data.pop("accelx")
    data.pop("accely")
    data.pop("accelz")
    data.pop("compassx")
    data.pop("compassy")
    data.pop("compassz")
    return np.array(data.copy())

inputs  = read_format_csv("./data_simple/imu1_10_27.csv")

print(inputs)





expected_outputs = None # TODO

[[ -1.482   0.866 -29.676]
 [ -1.477   0.894 -30.732]
 [ -1.47    0.895 -31.7  ]
 ...
 [  4.962   4.484 -94.003]
 [  5.183   4.539 -93.759]
 [  5.301   4.442 -93.834]]


In [ ]:
history = model.fit(
    inputs,
    expected_outputs,
    epochs = 5,
    validation_split = 0.2,
)

model.save("./")


In [ ]:
data_in = None
prediction = model.predict(data_in)

#TODO plot data_in and prediction